<a href="https://colab.research.google.com/github/youssefkamel02/AI-milestone/blob/main/milestone2_infosec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install argon2-cffi

In [ ]:
# ====================================================================
# BINF711 - Milestone 2: Cryptography and Encryption Techniques
# Complete Python Implementation (Playfair, Vigenère, and Argon2)
# ====================================================================

# NOTE: Requires external library: pip install argon2-cffi

try:
    from argon2 import PasswordHasher
    from argon2.exceptions import VerifyMismatchError
    # Flag to indicate if the real Argon2 library is available
    ARGON2_AVAILABLE = True
except ImportError:
    # --- Mock classes for environments without argon2-cffi ---
    class PasswordHasher:
        # Mock hash returns a string that lets us know it's a mock
        def hash(self, password): return f"$MOCK2id$v=19$m=65536,t=3,p=4${password[:8]}..."
        # Mock verify always raises an error if the password is wrong
        def verify(self, hashed_password, password):
            # Simple check to simulate correct/wrong verification
            if "wrong" in password:
                 raise VerifyMismatchError
            return True
        def check_needs_rehash(self, hashed_password): return False
    class VerifyMismatchError(Exception): pass
    ARGON2_AVAILABLE = False


# ------------------------------------
# 1. Playfair Cipher Implementation
# ------------------------------------

class PlayfairCipher:
    """Implements the Playfair Cipher. Note: 'J' is treated as 'I'."""
    ALPHABET = "ABCDEFGHIKLMNOPQRSTUVWXYZ"

    def __init__(self, key):
        self.key_table = self._generate_key_table(key.upper())

    def _generate_key_table(self, key):
        unique_key = ""
        processed_key = key.replace("J", "I")
        for char in processed_key:
            if 'A' <= char <= 'Z' and char not in unique_key:
                unique_key += char

        table_key_str = unique_key
        for char in self.ALPHABET:
            if char not in table_key_str:
                table_key_str += char

        key_table = []
        for i in range(5):
            row = list(table_key_str[i*5 : (i+1)*5])
            key_table.append(row)

        return key_table

    def _prepare_text(self, plaintext):
        prepared = "".join(c for c in plaintext.upper() if 'A' <= c <= 'Z').replace("J", "I")
        processed_text = ""
        i = 0
        while i < len(prepared):
            processed_text += prepared[i]

            if i + 1 < len(prepared):
                if prepared[i] == prepared[i+1]:
                    processed_text += 'X'
                else:
                    processed_text += prepared[i+1]
                    i += 1

            i += 1

        if len(processed_text) % 2 != 0:
            processed_text += 'X'

        return processed_text

    def _get_char_coords(self, char):
        for r in range(5):
            if char in self.key_table[r]:
                c = self.key_table[r].index(char)
                return r, c
        return -1, -1

    def encrypt(self, plaintext):
        prepared_text = self._prepare_text(plaintext)
        ciphertext = ""
        for i in range(0, len(prepared_text), 2):
            char1, char2 = prepared_text[i], prepared_text[i+1]
            r1, c1 = self._get_char_coords(char1)
            r2, c2 = self._get_char_coords(char2)

            if r1 == r2: # Same Row
                new_char1 = self.key_table[r1][(c1 + 1) % 5]
                new_char2 = self.key_table[r2][(c2 + 1) % 5]
            elif c1 == c2: # Same Column
                new_char1 = self.key_table[(r1 + 1) % 5][c1]
                new_char2 = self.key_table[(r2 + 1) % 5][c2]
            else: # Rectangle
                new_char1 = self.key_table[r1][c2]
                new_char2 = self.key_table[r2][c1]

            ciphertext += new_char1 + new_char2
        return ciphertext


# ------------------------------------
# 2. Vigenère Cipher Implementation
# ------------------------------------

class VigenereCipher:
    """Implements the Vigenère Cipher."""

    def __init__(self, key):
        self.key = self._clean_key(key)

    def _clean_text(self, text):
        return "".join(c for c in text.upper() if 'A' <= c <= 'Z')

    def _clean_key(self, key):
        cleaned_key = "".join(c for c in key.upper() if 'A' <= c <= 'Z')
        if not cleaned_key:
            raise ValueError("Vigenere key must contain at least one alphabetic character.")
        return cleaned_key

    def _generate_repeated_key(self, plaintext):
        cleaned_plaintext = self._clean_text(plaintext)
        key_length = len(self.key)
        full_repeats = len(cleaned_plaintext) // key_length
        remainder = len(cleaned_plaintext) % key_length
        repeated_key = (self.key * full_repeats) + self.key[:remainder]
        return repeated_key

    def encrypt(self, plaintext):
        cleaned_plaintext = self._clean_text(plaintext)
        repeated_key = self._generate_repeated_key(cleaned_plaintext)
        ciphertext = []

        for p_char, k_char in zip(cleaned_plaintext, repeated_key):
            p_val = ord(p_char) - ord('A')
            k_val = ord(k_char) - ord('A')
            c_val = (p_val + k_val) % 26
            c_char = chr(c_val + ord('A'))
            ciphertext.append(c_char)

        return "".join(ciphertext)


# ------------------------------------
# 3. Argon2 Password Hashing (Modern Technique)
# ------------------------------------

class Argon2PasswordManager:
    """Implements Argon2 for secure, one-way password hashing and verification."""

    def __init__(self):
        self.ph = PasswordHasher()

    def hash_password(self, password: str) -> str:
        return self.ph.hash(password)

    def verify_password(self, password: str, hashed_password: str) -> bool:
        """Correctly verifies the password, returning False on mismatch."""
        try:
            self.ph.verify(hashed_password, password)
            if self.ph.check_needs_rehash(hashed_password) and ARGON2_AVAILABLE:
                print("[INFO] Password hash needs update (rehash).")
            return True
        except VerifyMismatchError:
            return False
        except Exception as e:
            print(f"[ERROR] Verification failed due to internal error: {e}")
            return False


# ------------------------------------
# Main Application Runner
# ------------------------------------

def run_application():
    """Main function to run the password encryption demonstration."""

    print("===================================================================")
    print("       BINF711 - Milestone 2: Password Encryption Prototype")
    print("===================================================================")

    # Check for Argon2
    if not ARGON2_AVAILABLE:
        print("!! WARNING: Argon2 library not found. Using a MOCK implementation. !!")
        print("!! Run 'pip install argon2-cffi' for the actual required security. !!\n")

    # 1. Get common input
    password = input("Enter the password you wish to encrypt/hash: ")
    key = input("Enter a secret key (used for Playfair/Vigenere): ")

    if not key:
        print("\n[ERROR] Key cannot be empty. Exiting.")
        return

    # --- 1. Playfair Cipher Demonstration ---
    try:
        print("\n--- 1. Playfair Cipher ---")
        playfair = PlayfairCipher(key)
        playfair_ciphertext = playfair.encrypt(password)
        print(f"Prepared Text: {playfair._prepare_text(password)}")
        print(f"Ciphertext: {playfair_ciphertext}")
    except Exception as e:
        print(f"[ERROR] Playfair failed: {e}")

    # --- 2. Vigenère Cipher Demonstration ---
    try:
        print("\n--- 2. Vigenère Cipher ---")
        vigenere = VigenereCipher(key)
        vigenere_ciphertext = vigenere.encrypt(password)
        print(f"Ciphertext: {vigenere_ciphertext}")
    except Exception as e:
        print(f"[ERROR] Vigenère failed: {e}")

    # --- 3. Argon2 Password Hashing Demonstration (Modern) ---
    try:
        print("\n--- 3. Argon2 Hashing (Recommended Modern Method) ---")
        argon2_manager = Argon2PasswordManager()

        # Hashing
        argon2_hash = argon2_manager.hash_password(password)
        print(f"Generated Hash: {argon2_hash}")

        # Verification
        wrong_password = password + "z" # Creates a deliberately incorrect test password

        correct_check = argon2_manager.verify_password(password, argon2_hash)
        wrong_check = argon2_manager.verify_password(wrong_password, argon2_hash)

        print(f"Verification Check (Correct Password): {'SUCCESS' if correct_check else 'FAIL'}")
        print(f"Verification Check (Wrong Password): {'SUCCESS' if wrong_check else 'FAIL'}")

    except Exception as e:
        print(f"[ERROR] Argon2 failed. Check library installation: {e}")


if __name__ == "__main__":
    run_application()

       BINF711 - Milestone 2: Password Encryption Prototype
Enter the password you wish to encrypt/hash: youssef
Enter a secret key (used for Playfair/Vigenere): alo

--- 1. Playfair Cipher ---
Prepared Text: YOUSSEFX
Ciphertext: XBQTRFMO

--- 2. Vigenère Cipher ---
Ciphertext: YZISDSF

--- 3. Argon2 Hashing (Recommended Modern Method) ---
Generated Hash: $argon2id$v=19$m=65536,t=3,p=4$OjVLFH6iZr5ute+b94jHsA$Tm2dDfxsx8yvtYkS+KsAHUtGEZUfVgkGjQIaT07Hbqg
Verification Check (Correct Password): SUCCESS
Verification Check (Wrong Password): FAIL
